<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week08/Dependency_Injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **의존성주입(Dependency Injection)**
-  객체 지향 프로그래밍에서 사용되는 디자인 패턴 중 하나로, 객체 간의 의존성을 내부에서 직접 생성하는 것이 아니라 외부에서 주입받는 방식
- **느슨한 결합(Loose Coupling)** 을 달성하여 코드의 유연성, 재사용성, 테스트 용이성을 극대화
- 객체가 자신이 필요한 다른 객체(의존성)를 직접 생성하거나 찾는 대신, 외부에서 그 객체를 받아서 사용하게 만드는 디자인 패턴
## 1. 쿼리 매개변수 (Query Parameters)

In [2]:
def common_parameters(q, skip, limit):
    """
    공통적으로 사용될 쿼리 파라미터를 정의하고 딕셔너리로 묶어 반환하는 함수
    """
    return {"q": q, "skip": skip, "limit": limit}

In [3]:
common_parameters

<function __main__.common_parameters(q, skip, limit)>

In [4]:
common_parameters(q=None, skip=0, limit=10)

{'q': None, 'skip': 0, 'limit': 10}

In [5]:
def common_parameters(q: str | None = None, skip: int = 0, limit: int = 100):
    return {"q": q, "skip": skip, "limit": limit}

In [6]:
common_parameters

<function __main__.common_parameters(q: str | None = None, skip: int = 0, limit: int = 100)>

In [7]:
common_parameters(q=None, skip=0, limit=100)

{'q': None, 'skip': 0, 'limit': 100}

- Depends(common_parameters): FastAPI에게 read_items 함수를 실행하기 전에 common_parameters 함수를 먼저 실행해야 한다고 요청

- common_parameters 함수는 요청으로부터 q, skip, limit 쿼리 매개변수 값을 가져와 딕셔너리로 묶어 반환합니다.

- 이 반환된 딕셔너리가 commons라는 이름의 매개변수로 read_items 함수에 **주입(Injection)**
> - commons 변수(var)는
> - dict 데이터타입을 갖는
> - 실제 값은 common_parameters 함수의 결과 값
- read_items 함수는 주입받은 commons 딕셔너리 (즉, q, skip, limit 값)를 그대로 클라이언트에게 반환


```
from typing import Annotated : 구성 요소,역할
- Function(commons: Annotated[dict, Depends(common_parameters)])
- commons 변수의 **실제 타입 힌트는 dict**
- common_parameters 함수가 딕셔너리를 반환할 것
  . Depends(common_parameters),Annotated에 첨부된 메타데이터
  . FastAPI에게 이 변수의 값을 얻기 위해 Depends 함수(common_parameters)를 실행 요청
```


In [8]:
from typing import Annotated
from fastapi import Depends

def read_items(commons: Annotated[dict, Depends(common_parameters)]):
    return commons

In [9]:
read_items(commons={'q': 'some_query', 'skip': 5, 'limit': 20})

{'q': 'some_query', 'skip': 5, 'limit': 20}

In [10]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/items/")
async def read_items(commons: Annotated[dict, Depends(common_parameters)]):
    return commons

## FastAPI 실행
- **구글클라우드 환경이라 ngrok을 사용하여 외부 url생성하여 확인하기**

In [11]:
! pip install pyngrok -q

### **[ngrok API Key 발급받기](https://ngrok.com/)**
<img src='https://velog.velcdn.com/images/kya754/post/6890ccfe-7a49-40cb-bf99-d51807f8f450/68747470733a2f2f6e67726f6b2e636f6d2f7374617469632f696d672f6f766572766965772e706e67.png' width = 600>

In [1]:
from pyngrok import ngrok
import uvicorn
import os
import threading


In [24]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/items/")
async def read_items(commons: Annotated[dict, Depends(common_parameters)]):
    return commons

################### ngrok #########################
from google.colab import userdata
NGROK_API_KEY = userdata.get('ngrok')
# print(NGROK_API_KEY[::2])
ngrok.set_auth_token(NGROK_API_KEY)

def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000)

if __name__ == "__main__":
    # Open a HTTP tunnel on the default port 8000
    http_tunnel = ngrok.connect(8000)
    print(f"Public URL: {http_tunnel.public_url}")

    # Run uvicorn in a separate thread
    uvicorn_thread = threading.Thread(target=run_uvicorn)
    uvicorn_thread.start()

Public URL: https://37c3fb2e8210.ngrok-free.app


INFO:     Started server process [11833]


## 2. 데이터베이스

In [8]:
from pydantic import BaseModel, Field

class CustomerCreate(BaseModel):
    # Field(..., min_length=2)는 '필수'이며 최소 2자 이상이어야 함을 의미
    name: str = Field(..., min_length=2)
    # 이메일 형식 검사 및 필수
    email: str = Field(...)


def common_parameters(q: str | None = None, skip: int = 0, limit: int = 100):
    return {"q": q, "skip": skip, "limit": limit}

In [9]:
customer_data = {
    "name": "John Doe",
    "email": "john.doe@example.com"
}
customer: CustomerCreate = CustomerCreate(**customer_data)

display(customer)

CustomerCreate(name='John Doe', email='john.doe@example.com')

In [10]:
from fastapi import FastAPI

app = FastAPI()

fake_db = []
next_id = 1

@app.post("/customers/")
async def create_customer(customer: CustomerCreate):
    """
    POST 요청의 JSON 본문이 CustomerCreate 모델에 맞게 자동 검증됩니다.
    """
    global next_id

    # Pydantic 객체의 값에 접근하여 새 데이터 생성
    new_customer = customer.model_dump()
    new_customer["id"] = next_id
    fake_db.append(new_customer)
    next_id += 1

    # 응답
    return {"message": "고객 생성 성공", "customer": new_customer}


################### ngrok #########################
from google.colab import userdata
NGROK_API_KEY = userdata.get('ngrok')
# print(NGROK_API_KEY[::2])
ngrok.set_auth_token(NGROK_API_KEY)

def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000)

if __name__ == "__main__":
    # Open a HTTP tunnel on the default port 8000
    http_tunnel = ngrok.connect(8000)
    print(f"Public URL: {http_tunnel.public_url}")

    # Run uvicorn in a separate thread
    uvicorn_thread = threading.Thread(target=run_uvicorn)
    uvicorn_thread.start()

Public URL: https://b316bb3df508.ngrok-free.app


INFO:     Started server process [14312]


In [ ]:
!fuser -k 8000/tcp

## 3. 데이터베이스

```
# 1단계: 사용자가 API 호출
 - GET /users/

# 2단계: FastAPI가 get_users 함수를 실행하기 전에 Depends(get_db)를 발견

# 3단계: get_db() 함수 실행
def get_db():
    conn = sqlite3.connect("test.db")  # 연결 생성
    yield conn  # 연결을 반환
    conn.close()  # 나중에 자동 실행

# 4단계: get_db()가 반환한 conn을 db 매개변수에 주입
async def get_users(db: sqlite3.Connection):  # db = conn
    cursor = db.cursor()  # 주입받은 연결 사용
    # ...

# 5단계: 응답 반환 후 get_db()의 finally 블록 실행 (자동 정리)

```

### **db: Annotated[sqlite3.Connection, Depends(get_db)]**
#### ❌ 잘못된 이해
db: conn  # conn은 '값(변수)'이지 '타입'이 아님!

#### ✅ 올바른 작성
db: sqlite3.Connection  # Connection은 '타입(클래스)'

In [3]:
from fastapi import FastAPI, Depends
from typing import Annotated
import sqlite3
from contextlib import contextmanager

app = FastAPI()

# 데이터베이스 파일 경로
DATABASE = "test.db"

# 1. 데이터베이스 연결을 제공하는 의존성 함수
def get_db():
    """
    데이터베이스 연결을 생성하고 요청 후 자동으로 닫아주는 제너레이터 함수
    """
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row  # 딕셔너리처럼 접근 가능
    try:
        yield conn  # 연결을 반환
    finally:
        conn.close()  # 요청 완료 후 자동으로 닫힘


# 2. 테이블 초기화 함수
def init_db():
    """데이터베이스 테이블 생성"""
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    # users 테이블 생성
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL
        )
    """)

    conn.commit()
    conn.close()


# 3. API 엔드포인트 - 사용자 조회 (의존성 주입 사용)
@app.get("/users/")
async def get_users(db: Annotated[sqlite3.Connection, Depends(get_db)]):
    """
    모든 사용자 조회
    db: get_db() 함수에서 주입받은 데이터베이스 연결
    """
    cursor = db.cursor()
    cursor.execute("SELECT * FROM users")
    users = cursor.fetchall()

    # sqlite3.Row를 딕셔너리로 변환
    return [dict(user) for user in users]


# 4. API 엔드포인트 - 사용자 생성 (의존성 주입 사용)
@app.post("/users/")
async def create_user(
    name: str,
    email: str,
    db: Annotated[sqlite3.Connection, Depends(get_db)]
):
    """
    새 사용자 생성
    db: get_db() 함수에서 주입받은 데이터베이스 연결
    """
    cursor = db.cursor()

    try:
        cursor.execute(
            "INSERT INTO users (name, email) VALUES (?, ?)",
            (name, email)
        )
        db.commit()

        # 생성된 사용자 ID 가져오기
        user_id = cursor.lastrowid

        return {
            "message": "사용자 생성 성공",
            "user": {"id": user_id, "name": name, "email": email}
        }
    except sqlite3.IntegrityError:
        return {"error": "이미 존재하는 이메일입니다"}


# 5. API 엔드포인트 - 특정 사용자 조회
@app.get("/users/{user_id}")
async def get_user(
    user_id: int,
    db: Annotated[sqlite3.Connection, Depends(get_db)]
):
    """
    ID로 특정 사용자 조회
    """
    cursor = db.cursor()
    cursor.execute("SELECT * FROM users WHERE id = ?", (user_id,))
    user = cursor.fetchone()

    if user:
        return dict(user)
    return {"error": "사용자를 찾을 수 없습니다"}


# 6. API 엔드포인트 - 사용자 삭제
@app.delete("/users/{user_id}")
async def delete_user(
    user_id: int,
    db: Annotated[sqlite3.Connection, Depends(get_db)]
):
    """
    사용자 삭제
    """
    cursor = db.cursor()
    cursor.execute("DELETE FROM users WHERE id = ?", (user_id,))
    db.commit()

    if cursor.rowcount > 0:
        return {"message": f"사용자 ID {user_id} 삭제 완료"}
    return {"error": "사용자를 찾을 수 없습니다"}


# 앱 시작 시 데이터베이스 초기화
@app.on_event("startup")
async def startup_event():
    """서버 시작 시 실행"""
    init_db()
    print("데이터베이스 초기화 완료")


# 실행 방법: vscod에서
# uvicorn filename:app --reload
#
# 테스트 URL:
# - GET  http://localhost:8000/users/
# - POST http://localhost:8000/users/?name=홍길동&email=hong@example.com
# - GET  http://localhost:8000/users/1
# - DELETE http://localhost:8000/users/1

/tmp/ipython-input-2816802298.py:126: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


In [5]:
from pyngrok import ngrok
import uvicorn
import os
import threading

################### ngrok #########################
from google.colab import userdata
NGROK_API_KEY = userdata.get('ngrok')
# print(NGROK_API_KEY[::2])
ngrok.set_auth_token(NGROK_API_KEY)

def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000)

if __name__ == "__main__":
    # Open a HTTP tunnel on the default port 8000
    http_tunnel = ngrok.connect(8000)
    print(f"Public URL: {http_tunnel.public_url}")

    # Run uvicorn in a separate thread
    uvicorn_thread = threading.Thread(target=run_uvicorn)
    uvicorn_thread.start()

Public URL: https://a2ede6aaf41e.ngrok-free.app


### 아래 코드는 수정된 코드 (Optional)
- /tmp/ipython-input-2816802298.py:126: DeprecationWarning:
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")

In [2]:
from fastapi import FastAPI, Depends
from typing import Annotated
import sqlite3
from contextlib import asynccontextmanager

# 데이터베이스 파일 경로
DATABASE = "test.db"

# 1. 데이터베이스 연결을 제공하는 의존성 함수
def get_db():
    """
    데이터베이스 연결을 생성하고 요청 후 자동으로 닫아주는 제너레이터 함수
    """
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row  # 딕셔너리처럼 접근 가능
    try:
        yield conn  # 연결을 반환
    finally:
        conn.close()  # 요청 완료 후 자동으로 닫힘


# 2. 테이블 초기화 함수
def init_db():
    """데이터베이스 테이블 생성"""
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    # users 테이블 생성
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL
        )
    """)

    conn.commit()
    conn.close()

# Lifespan event handler
@asynccontextmanager
async def lifespan(app: FastAPI):
    """서버 시작 및 종료 시 실행될 이벤트 핸들러"""
    print("데이터베이스 초기화 시작")
    init_db()
    print("데이터베이스 초기화 완료")
    yield


app = FastAPI(lifespan=lifespan)


# 3. API 엔드포인트 - 사용자 조회 (의존성 주입 사용)
@app.get("/users/")
async def get_users(db: Annotated[sqlite3.Connection, Depends(get_db)]):
    """
    모든 사용자 조회
    db: get_db() 함수에서 주입받은 데이터베이스 연결
    """
    cursor = db.cursor()
    cursor.execute("SELECT * FROM users")
    users = cursor.fetchall()

    # sqlite3.Row를 딕셔너리로 변환
    return [dict(user) for user in users]


# 4. API 엔드포인트 - 사용자 생성 (의존성 주입 사용)
@app.post("/users/")
async def create_user(
    name: str,
    email: str,
    db: Annotated[sqlite3.Connection, Depends(get_db)]
):
    """
    새 사용자 생성
    db: get_db() 함수에서 주입받은 데이터베이스 연결
    """
    cursor = db.cursor()

    try:
        cursor.execute(
            "INSERT INTO users (name, email) VALUES (?, ?)",
            (name, email)
        )
        db.commit()

        # 생성된 사용자 ID 가져오기
        user_id = cursor.lastrowid

        return {
            "message": "사용자 생성 성공",
            "user": {"id": user_id, "name": name, "email": email}
        }
    except sqlite3.IntegrityError:
        return {"error": "이미 존재하는 이메일입니다"}


# 5. API 엔드포인트 - 특정 사용자 조회
@app.get("/users/{user_id}")
async def get_user(
    user_id: int,
    db: Annotated[sqlite3.Connection, Depends(get_db)]
):
    """
    ID로 특정 사용자 조회
    """
    cursor = db.cursor()
    cursor.execute("SELECT * FROM users WHERE id = ?", (user_id,))
    user = cursor.fetchone()

    if user:
        return dict(user)
    return {"error": "사용자를 찾을 수 없습니다"}


# 6. API 엔드포인트 - 사용자 삭제
@app.delete("/users/{user_id}")
async def delete_user(
    user_id: int,
    db: Annotated[sqlite3.Connection, Depends(get_db)]
):
    """
    사용자 삭제
    """
    cursor = db.cursor()
    cursor.execute("DELETE FROM users WHERE id = ?", (user_id,))
    db.commit()

    if cursor.rowcount > 0:
        return {"message": f"사용자 ID {user_id} 삭제 완료"}
    return {"error": "사용자를 찾을 수 없습니다"}



